In [3]:
import pandas as pd
import re
import concurrent.futures
import requests
import openai
from tqdm import tqdm
tqdm.pandas()

In [2]:
def cleanhtml(raw_html):
    CLEANR = re.compile('<.*?>')
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext
    
def clean_data(df):
    df["pt_answer"] = df["pt_answer"].apply(lambda x: cleanhtml(x))

    df["question"] = df["pt_title"].str.lower() + "\n" + df["pt_body"]
    df["question"] = df["question"].apply(lambda x: cleanhtml(x))
    df["answer"] = df["pt_answer"].str.lower()

    df = df[['pt_post_id','question', 'answer']]
    return df
    

## get qa and link to post
def get_url(df):
    url = []
    for index, row in df.iterrows():
        url.append(f"https://stackoverflow.com/questions/{row['pt_post_id']}/")
    
    df['source'] = url

    return df

### Download the file pt_question_answers.csv from https://github.com/chauhang/pt-experiments/blob/pytorch-qa/pytorch-qa/pt_question_answers.csv

In [3]:
df = pd.read_csv('pt_question_answers.csv')
df.shape

(10763, 12)

In [4]:
df = clean_data(df)
df = get_url(df)
df = df[['question','answer','source']]

In [5]:
df = df[:5000]

df.shape

(5000, 3)

### Set your openai key here

In [6]:
OPENAI_API_KEY=""

In [7]:
def get_qa_openai(prompt, index):
    try:
        completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo", api_key = OPENAI_API_KEY,
                messages=prompt
                )

        resp = completion.choices[0].message.content

    except requests.exceptions.RequestException as e:
        print(f'Request failed with error: {str(e)}.')
        print(f'Waiting for 3 minutes before trying again...')
        time.sleep(180)
    
    return (resp, index)

In [172]:
questions_ans = []

with concurrent.futures.ThreadPoolExecutor() as executor:
    
    futures = []
    for index,i in df.iterrows():

        question = i["question"]
        answer = i["answer"]
    
        prompt = [{"role":"system","content":"you are an intelligent system who has the skills of an pytorch engineer. take the question and answer as input and generate an answer for the question, include code snippets if necessary."},
                  {"role":"assistant","content":"sure, give me the input question and answer"},
                  {"role":"user","content":f"question: {question}, answer:{answer}"}]

        futures.append(executor.submit(get_qa_openai, prompt, index))

    for future, (_, row) in tqdm(zip(concurrent.futures.as_completed(futures), df.iterrows()), total=len(df)):
        try:
            qa, ind = future.result()
            questions_ans.append((ind,qa))
        except Exception as exc:
            print(f'generated an exception: {exc}')

  1%|▌                                        | 71/5000 [00:26<21:52,  3.75it/s]

generated an exception: This model's maximum context length is 4097 tokens. However, your messages resulted in 4747 tokens. Please reduce the length of the messages.


  5%|█▊                                      | 228/5000 [01:19<32:41,  2.43it/s]

generated an exception: The server is overloaded or not ready yet.


  5%|██▏                                     | 274/5000 [01:38<26:07,  3.01it/s]

...

 99%|██████████████████████████████████████▊| 4971/5000 [29:36<00:01, 17.54it/s]

generated an exception: Rate limit reached for default-gpt-3.5-turbo in organization org-oXRtqZdWlsnmtry277X001Hu on tokens per min. Limit: 90000 / min. Current: 89720 / min. Contact us through our help center at help.openai.com if you continue to have issues.
generated an exception: Rate limit reached for default-gpt-3.5-turbo in organization org-oXRtqZdWlsnmtry277X001Hu on tokens per min. Limit: 90000 / min. Current: 89697 / min. Contact us through our help center at help.openai.com if you continue to have issues.
generated an exception: Rate limit reached for default-gpt-3.5-turbo in organization org-oXRtqZdWlsnmtry277X001Hu on tokens per min. Limit: 90000 / min. Current: 89680 / min. Contact us through our help center at help.openai.com if you continue to have issues.
generated an exception: Rate limit reached for default-gpt-3.5-turbo in organization org-oXRtqZdWlsnmtry277X001Hu on tokens per min. Limit: 90000 / min. Current: 89594 / min. Contact us through our help center at help

100%|██████████████████████████████████████▊| 4976/5000 [29:38<00:04,  5.60it/s]

generated an exception: The server is overloaded or not ready yet.


100%|███████████████████████████████████████| 5000/5000 [30:00<00:00,  2.78it/s]


In [176]:
for index, qa in questions_ans:
    df.at[index, 'short_answer'] = qa

In [47]:
df

,question,answer,source,short_answer
0,extracting the top-k value-indices from a 1-d ...,as of pull request #496 torch now includes a b...,https://stackoverflow.com/questions/34750268/,To extract the indices of the top-k values in ...
1,how to display custom images in tensorboard (e...,it is quite easy to do if you have the image i...,https://stackoverflow.com/questions/38543850/,The code provided in the answer is a direct wa...
2,python wheels: cp27mu not supported\nI'm tryin...,this is exactly that. \nrecompile python under...,https://stackoverflow.com/questions/41767005/,To recompile Python with `--enable-unicode=ucs...
3,loading torch7 trained models (.t7) in pytorch...,as of pytorch 1.0 torch.utils.serialization is...,https://stackoverflow.com/questions/41861354/,To load torch7 trained models (.t7) in PyTorch...
4,pytorch: how to use dataloaders for custom dat...,"yes, that is possible. just create the objects...",https://stackoverflow.com/questions/41924453/,Great question! Here's an answer with code sni...
...,...,...,...,...
4995,compare two segmentation maps predictions\nI a...,"yes, this is a valid way to implement consiste...",https://stackoverflow.com/questions/67682106/,NaN
4996,difference between dataset and tensordataset i...,the dataset class is an abstract class that is...,https://stackoverflow.com/questions/67683406/,NaN
4997,define nn.parameters with a for loop\nI am int...,best way to accomplish this you can accomplish...,https://stackoverflow.com/questions/67689104/,NaN
4998,how to integrate a pytorch model into a dynami...,tensorflow or pytorch models can't be directly...,https://stackoverflow.com/questions/67693181/,NaN


In [45]:
df.to_csv("answer_summarized_5000.csv", index=False)